In [21]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)


examples = [
    {
        "question": "What do you know about Titanic?",
        "answer":"""
        Here is what I know:
        Director: James Cameron
        Main Cast: Leonardo DiCaprio (Jack Dawson), Kate Winslet (Rose DeWitt Bukater), Billy Zane (Cal Hockley)
        Budget: $200 million
        Box Office Revenue: $2.195 billion
        Genre: Romance, Drama, Disaster
        Quick Synopsis: Titanic is a love story set against the ill-fated maiden voyage of the R.M.S. Titanic. Jack, a poor artist, and Rose, an aristocratic young woman, fall in love aboard the ship despite their different social standings. The film portrays their romance, the tragic sinking of the ship, and the struggles for survival as the disaster unfolds.
        """ 
    },
    {
        "question": "What do you know about  Spider-Man 3?",
        "answer":"""
        Here is what I know:
        Director: Sam Raimi
        Main Cast: Tobey Maguire (Peter Parker / Spider-Man), Kirsten Dunst (Mary Jane Watson), James Franco (Harry Osborn), Thomas Haden Church (Flint Marko / Sandman), Topher Grace (Eddie Brock / Venom)
        Budget: $258 million
        Box Office Revenue: $894.9 million
        Genre: Superhero, Action, Fantasy
        Quick Synopsis: Spider-Man 3 follows Peter Parker as he struggles with the responsibilities of being Spider-Man, his relationship with Mary Jane, and new threats, including the vengeful Harry Osborn, who takes on the identity of the New Goblin, the Sandman, who has a tragic connection to Peter's past, and the emergence of the dark and powerful alien symbiote that transforms Peter into a more aggressive version of himself. The film explores themes of redemption, forgiveness, and the internal battle between good and evil.
        """ 
    },
    {
        "question": "What do you know about Parasite?",
        "answer":"""
        Here is what I know:
        Director: Bong Joon-ho
        Main Cast: Song Kang-ho (Kim Ki-taek), Lee Sun-kyun (Park Dong-ik), Cho Yeo-jeong (Choi Yeon-gyo), Choi Woo-shik (Kim Ki-woo), Park So-dam (Kim Ki-jung)
        Budget: $11 million
        Box Office Revenue: $263.1 million
        Genre: Thriller, Drama, Dark Comedy
        Quick Synopsis: Parasite is a critically acclaimed South Korean film that explores class disparities and social inequalities. The story follows the poor Kim family, who scheme to infiltrate the wealthy Park family's household by posing as unrelated, highly qualified individuals. As the Kims take over various jobs within the Park family, their deception leads to unexpected and increasingly dangerous consequences, culminating in a dramatic and thought-provoking commentary on society's divisions.
        """  
    },  
]

# ChatPromptTemplate.from_messages sets up the prompt template by automatically handling variables like {movie} and {answer} within the structured messages.
manual_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

# FewShotChatMessagePromptTemplate: Helps format the LLM's output by providing examples, guiding it to mimic the structure and style of the provided data.
learning_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = manual_example_prompt,
    examples=examples   
)

# Finally, with the few-shot prompt (informed by the examples), we ask the question.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert"),
    learning_prompt,
    ("human", "What do you know about {movie}?"),
])

chain = final_prompt | chat

chain.invoke({"movie": "School Of Rock"})


        Here is what I know:
        Director: Richard Linklater
        Main Cast: Jack Black (Dewey Finn), Joan Cusack (Rosalie Mullins), Mike White (Ned Schneebly), Sarah Silverman (Patty Di Marco)
        Budget: $35 million
        Box Office Revenue: $131.3 million
        Genre: Comedy, Music
        Quick Synopsis: School of Rock is a feel-good comedy film about Dewey Finn, a struggling rock musician who poses as a substitute teacher at a prestigious prep school. Instead of teaching traditional subjects, Dewey forms a rock band with his young students to compete in a Battle of the Bands competition. Through music, Dewey helps the students discover their talents, build confidence, and challenge the school's rigid academic culture. The film celebrates the power of music, creativity, and individuality.

AIMessageChunk(content="\n        Here is what I know:\n        Director: Richard Linklater\n        Main Cast: Jack Black (Dewey Finn), Joan Cusack (Rosalie Mullins), Mike White (Ned Schneebly), Sarah Silverman (Patty Di Marco)\n        Budget: $35 million\n        Box Office Revenue: $131.3 million\n        Genre: Comedy, Music\n        Quick Synopsis: School of Rock is a feel-good comedy film about Dewey Finn, a struggling rock musician who poses as a substitute teacher at a prestigious prep school. Instead of teaching traditional subjects, Dewey forms a rock band with his young students to compete in a Battle of the Bands competition. Through music, Dewey helps the students discover their talents, build confidence, and challenge the school's rigid academic culture. The film celebrates the power of music, creativity, and individuality.")

In [34]:
# LengthBasedExampleSelector
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 180,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human: What do you know about {movie}?",
    input_variables = ["movie"],
)

prompt.format(movie = "Parasite")

'Human: What do you know about Titanic?\nAI:\n        Here is what I know:\n        Director: James Cameron\n        Main Cast: Leonardo DiCaprio (Jack Dawson), Kate Winslet (Rose DeWitt Bukater), Billy Zane (Cal Hockley)\n        Budget: $200 million\n        Box Office Revenue: $2.195 billion\n        Genre: Romance, Drama, Disaster\n        Quick Synopsis: Titanic is a love story set against the ill-fated maiden voyage of the R.M.S. Titanic. Jack, a poor artist, and Rose, an aristocratic young woman, fall in love aboard the ship despite their different social standings. The film portrays their romance, the tragic sinking of the ship, and the struggles for survival as the disaster unfolds.\n        \n\nHuman: What do you know about Parasite?'

In [39]:
# Create custom selector for movies
from typing import Dict, List
from langchain.prompts.example_selector.base import BaseExampleSelector

class GenreBasedMovieExampleSelector(BaseExampleSelector):

    def __init__(self, examples, genre):
        self.examples = examples
        self.genre = genre
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        input_genres = self.genre.split(", ")
        
        filtered_examples = [
            example for example in self.examples
            if any(genre.lower() in example.get("answer", "").lower() for genre in input_genres)
        ]

        if not filtered_examples:
            print(f"No examples found for genres: {self.genre}")
            return []
        
        return filtered_examples
    
example_selector = GenreBasedMovieExampleSelector(
    examples = examples,
    genre = "Comedy, romance",
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human: What do you know about {movie}?",
    input_variables = ["movie"],
)

prompt.format(movie = "Parasite")

"Human: What do you know about Titanic?\nAI:\n        Here is what I know:\n        Director: James Cameron\n        Main Cast: Leonardo DiCaprio (Jack Dawson), Kate Winslet (Rose DeWitt Bukater), Billy Zane (Cal Hockley)\n        Budget: $200 million\n        Box Office Revenue: $2.195 billion\n        Genre: Romance, Drama, Disaster\n        Quick Synopsis: Titanic is a love story set against the ill-fated maiden voyage of the R.M.S. Titanic. Jack, a poor artist, and Rose, an aristocratic young woman, fall in love aboard the ship despite their different social standings. The film portrays their romance, the tragic sinking of the ship, and the struggles for survival as the disaster unfolds.\n        \n\nHuman: What do you know about Parasite?\nAI:\n        Here is what I know:\n        Director: Bong Joon-ho\n        Main Cast: Song Kang-ho (Kim Ki-taek), Lee Sun-kyun (Park Dong-ik), Cho Yeo-jeong (Choi Yeon-gyo), Choi Woo-shik (Kim Ki-woo), Park So-dam (Kim Ki-jung)\n        Budget: 